In [1]:
#encoding=utf8
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SparkSession

In [2]:

spark = SparkSession.builder.appName('spark_sql_demo').getOrCreate()
sc = spark.sparkContext



In [3]:
ssc = StreamingContext(sc, 5)
propertites={
#     'bootstrap.servers':'192.168.1.104:9092',
    "auto.offset.reset": "largest" #这个是错的但是并没有报错，说明ssc应该是没有运行的
}
kafka_strem_context = KafkaUtils.createStream(ssc,"192.168.1.104:2181","streaming_consumer_001",{"streaming_demo01":1},kafkaParams=propertites)


# ssc = StreamingContext(sc, 1)
# propertites={
# #     'bootstrap.servers':'192.168.1.104:9092',
#     "auto.offset.reset": "largest"
# }
# kafka_strem_context = KafkaUtils.createStream(ssc,"192.168.1.104:2181","streaming_consumer_001",{"streaming_demo01":1})



In [4]:
ss = kafka_strem_context.map(lambda x: x[1]).map(lambda x: (x.split("||")[0],x.split("||")[1])).reduceByKey(lambda a,b:a+b)

In [5]:
print(kafka_strem_context.pprint()) #为什么一直没输出呢？难道jupyter不适合做流数据处理？

None


In [6]:
print(ss.pprint())

None


In [7]:
def toredis(rdd):
    import redis
    rclient = redis.Redis(host="127.0.0.1", port=6379)
    for y in rdd:
        print(y)
        print(y[0],y[1])
        rclient.incrby(y[0],y[1])

In [8]:
toredis([["qqq",20]])

['qqq', 20]
qqq 20


In [9]:
kafka_strem_context.map(lambda x: x[1]).map(lambda x: (x.split("||")[0],x.split("||")[1])).reduceByKey(lambda a,b:a+b).foreachRDD(lambda q:q.foreachPartition(toredis))

In [11]:
ssc.start()#必须使用这个显式启动才能有效果
#运行报错： java.lang.NoClassDefFoundError: org/apache/spark/internal/Logging$class

In [ ]:
ssc.awaitTermination()
sparkcontext.stop()